# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [1]:
import pandas as pd
import requests
import numpy as np

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
# 2) seu código aqui
(sinasc
    .isna()
    .sum()
)

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [3]:
# 3) seu código aqui
cols = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
df = sinasc[cols].copy()
(df
    .isna()
    .sum()
)

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27028 entries, 0 to 27027
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LOCNASC     27028 non-null  int64  
 1   IDADEMAE    27028 non-null  int64  
 2   ESTCIVMAE   26711 non-null  float64
 3   ESCMAE      26716 non-null  object 
 4   QTDFILVIVO  25455 non-null  float64
 5   GESTACAO    25796 non-null  object 
 6   GRAVIDEZ    26949 non-null  object 
 7   CONSULTAS   27028 non-null  int64  
 8   APGAR5      26925 non-null  float64
dtypes: float64(3), int64(3), object(3)
memory usage: 1.9+ MB


In [5]:
# 4) seu código aqui
df = df.dropna(subset=['APGAR5'])

In [6]:
df['ESTCIVMAE'].value_counts(dropna=False)

ESTCIVMAE
2.0    9906
1.0    8907
5.0    7145
4.0     451
NaN     315
9.0     144
3.0      57
Name: count, dtype: int64

In [7]:
# 5) seu código aqui
df['ESTCIVMAE'] = df['ESTCIVMAE'].fillna(9)
df

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [8]:
(df['ESTCIVMAE']
    .isna()
    .sum()
)

0

In [9]:
df['CONSULTAS'] = df['CONSULTAS'].fillna(9)
df

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [10]:
(df['CONSULTAS']
    .isna()
    .sum()
)

0

In [11]:
# 6) Seu código aqui
df['QTDFILVIVO'] = df['QTDFILVIVO'].fillna(0)
df

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [12]:
(df['QTDFILVIVO']
    .isna()
    .sum()
)

0

In [13]:
# 7) seu código aqui
print(df.info())
df.loc[df.isna().any(axis=1)]

<class 'pandas.core.frame.DataFrame'>
Index: 26925 entries, 0 to 27027
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LOCNASC     26925 non-null  int64  
 1   IDADEMAE    26925 non-null  int64  
 2   ESTCIVMAE   26925 non-null  float64
 3   ESCMAE      26615 non-null  object 
 4   QTDFILVIVO  26925 non-null  float64
 5   GESTACAO    25709 non-null  object 
 6   GRAVIDEZ    26849 non-null  object 
 7   CONSULTAS   26925 non-null  int64  
 8   APGAR5      26925 non-null  float64
dtypes: float64(3), int64(3), object(3)
memory usage: 2.1+ MB
None


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
38,1,25,1.0,NaN,3.0,37 a 41 semanas,Única,4,8.0
62,1,20,9.0,NaN,1.0,37 a 41 semanas,Única,4,9.0
66,1,23,5.0,NaN,1.0,37 a 41 semanas,Única,4,10.0
93,1,22,5.0,NaN,2.0,37 a 41 semanas,Única,3,9.0
229,1,25,1.0,NaN,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
26800,1,24,5.0,8 a 11 anos,1.0,NaN,Única,3,10.0
26855,1,25,5.0,8 a 11 anos,2.0,NaN,Única,3,10.0
26902,1,22,5.0,NaN,1.0,37 a 41 semanas,Única,3,10.0
26928,1,36,1.0,NaN,3.0,32 a 36 semanas,Dupla,3,9.0


In [14]:
# 8) Seu código aqui
df['ESCMAE'].value_counts(normalize=True, dropna=False)

ESCMAE
8 a 11 anos        0.578682
12 anos ou mais    0.213816
4 a 7 anos         0.175562
1 a 3 anos         0.018942
NaN                0.011513
Nenhuma            0.001486
Name: proportion, dtype: float64

In [15]:
df['ESCMAE'] = df['ESCMAE'].fillna('Ignorado') 
df['ESCMAE'].value_counts(dropna=False)

ESCMAE
8 a 11 anos        15581
12 anos ou mais     5757
4 a 7 anos          4727
1 a 3 anos           510
Ignorado             310
Nenhuma               40
Name: count, dtype: int64

In [16]:
df['GESTACAO'].value_counts(normalize=True, dropna=False)

GESTACAO
37 a 41 semanas        0.836509
32 a 36 semanas        0.082971
NaN                    0.045162
42 semanas e mais      0.025924
28 a 31 semanas        0.006017
22 a 27 semanas        0.003157
Menos de 22 semanas    0.000260
Name: proportion, dtype: float64

In [17]:
df['GESTACAO'] = df['GESTACAO'].fillna('37 a 41 semanas')
df['GESTACAO'].value_counts(dropna=False)

GESTACAO
37 a 41 semanas        23739
32 a 36 semanas         2234
42 semanas e mais        698
28 a 31 semanas          162
22 a 27 semanas           85
Menos de 22 semanas        7
Name: count, dtype: int64

In [18]:
df['GRAVIDEZ'].value_counts(normalize=True, dropna=False)

GRAVIDEZ
Única              0.978867
Dupla              0.018124
NaN                0.002823
Tríplice e mais    0.000186
Name: proportion, dtype: float64

In [19]:
df['GRAVIDEZ'] = df['GRAVIDEZ'].fillna('Única')
df['GRAVIDEZ'].value_counts(dropna=False)

GRAVIDEZ
Única              26432
Dupla                488
Tríplice e mais        5
Name: count, dtype: int64

#### Justificativa: verifiquei nas colunas restantes se havia um valor mais provável analisando a proporção das classes e fiz a substituição dos missing pelo valor retornado referente a mesma. Com exceção da escolaridade, cuja diferença entre as proporções não foi tão grande e, por isso, os NaN foram passados como código 9: 'Ignorado'.

In [20]:
# 9) seu código aqui
df.loc[df['APGAR5'] <= 3, 'APGAR_CAT'] = 'asfixia severa'
df.loc[(df['APGAR5'] > 3) & (df['APGAR5'] <= 5), 
    'APGAR_CAT'] = 'asfixia moderada'
df.loc[(df['APGAR5'] > 5) & (df['APGAR5'] <= 7),
    'APGAR_CAT'] = 'asfixia leve'
df.loc[df['APGAR5'] >= 8, 'APGAR_CAT'] = 'normal'

In [21]:
df['APGAR_CAT'].value_counts()

APGAR_CAT
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64

In [22]:
df

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,APGAR_CAT
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal


In [23]:
snake_case_cols = {
    'LOCNASC':'loc_nasc',
    'IDADEMAE':'idade_mae',
    'ESTCIVMAE':'est_civ_mae',
    'ESCMAE':'esc_mae',
    'QTDFILVIVO':'qtd_fil_vivo',
    'GESTACAO':'gestacao',
    'GRAVIDEZ':'gravidez',
    'CONSULTAS':'consultas',
    'APGAR5':'apgar_5',
    'APGAR_CAT':'apgar_cat'
}
df = df.rename(columns=snake_case_cols)

In [24]:
df.reset_index(drop=True)

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar_5,apgar_cat
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
26920,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
26921,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
26922,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
26923,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal


In [25]:
df_teste = df.copy()

In [26]:
faixas = [-1, 3, 5, 7, 10]
cat = ['asfixia severa', 'asfixia moderada', 'asfixia leve', 'normal']

df_teste['cat_apgar_cut'] = pd.cut(df_teste['apgar_5'], bins=faixas, labels=cat)
df_teste.head()

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar_5,apgar_cat,cat_apgar_cut
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal,normal


In [27]:
df_teste[['apgar_5','cat_apgar_cut']].value_counts(dropna=False).sort_index()

apgar_5  cat_apgar_cut   
0.0      asfixia severa         17
1.0      asfixia severa         24
2.0      asfixia severa         18
3.0      asfixia severa         15
4.0      asfixia moderada       23
5.0      asfixia moderada       45
6.0      asfixia leve           79
7.0      asfixia leve          241
8.0      normal               1994
9.0      normal              16076
10.0     normal               8393
Name: count, dtype: int64

In [28]:
conds = [
    df_teste['apgar_5'] <= 3,
    df_teste['apgar_5'] <= 5,
    df_teste['apgar_5'] <= 7,
    df_teste['apgar_5'] <= 10
]
cat = ['asfixia severa', 'asfixia moderada', 'asfixia leve', 'normal']
df_teste['cat_apgar_select'] = np.select(conds, cat)

In [29]:
df_teste[['apgar_5','cat_apgar_cut','cat_apgar_select']].value_counts(dropna=False).sort_index()

apgar_5  cat_apgar_cut     cat_apgar_select
0.0      asfixia severa    asfixia severa         17
1.0      asfixia severa    asfixia severa         24
2.0      asfixia severa    asfixia severa         18
3.0      asfixia severa    asfixia severa         15
4.0      asfixia moderada  asfixia moderada       23
5.0      asfixia moderada  asfixia moderada       45
6.0      asfixia leve      asfixia leve           79
7.0      asfixia leve      asfixia leve          241
8.0      normal            normal               1994
9.0      normal            normal              16076
10.0     normal            normal               8393
Name: count, dtype: int64

In [30]:
df_teste

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar_5,apgar_cat,cat_apgar_cut,cat_apgar_select
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal,normal,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal,normal,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal,normal,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal,normal,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal,normal,normal
...,...,...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal,normal,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal,normal,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal,normal,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal,normal,normal
